# 1. Load Text From TXT

## 1.1. Thư viện

In [1]:
from langchain_community.document_loaders import TextLoader

## 1.2. Mã nguồn

In [2]:
loader = TextLoader(r'data\tinh_huong\giaoduc_daotao_yte_data.txt', encoding='utf-8')

In [3]:
documents = loader.load()

# 2. Semantic Chunker

## 2.1. Thư viện

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain_experimental.text_splitter import SemanticChunker

In [5]:
from transformers import AutoModel, AutoTokenizer
import torch

class PhoBERTEmbeddings:
    def __init__(self, model_name):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def embed(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
    
    def embed_documents(self, texts):
        embeddings = []
        for text in texts:
            embeddings.append(self.embed(text))
        return embeddings

# Khởi tạo PhoBERT-large embedding
phobert_embedding = PhoBERTEmbeddings(model_name="vinai/phobert-large")


d:\Download_Python\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
BREAKPOINT_THRESHOLD_TYPE = os.getenv("BREAKPOINT_THRESHOLD_TYPE")

# Sử dụng trong SemanticChunker
text_splitter = SemanticChunker(embeddings=phobert_embedding, breakpoint_threshold_type=BREAKPOINT_THRESHOLD_TYPE)

In [7]:
docs = text_splitter.split_documents(documents)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


IndexError: index out of range in self